# Create and Register a Custom Template

```{article-info}
:author: Altay Sansal
:date: "{sub-ref}`today`"
:read-time: "{sub-ref}`wordcount-minutes` min read"
:class-container: sd-p-0 sd-outline-muted sd-rounded-3 sd-font-weight-light
```

```{warning}
Most SEG-Y files correspond to standard seismic data types or field configurations. We recommend using
the built-in templates from the registry whenever possible. Create a custom template only when your file
is unusual and cannot be represented by existing templates. In many cases, you can simply customize the
SEG-Y header byte mapping during ingestion without defining a new template.
```

In this tutorial we will walk through the Template Registry and show how to:

- Discover available templates in the registry
- Define and register your own template
- Generate a tiny synthetic SEG-Y file using the TGSAI/segy library
- Ingest that SEG-Y into MDIO using the custom template you defined

If this is your first time with MDIO, you may want to skim the Quickstart first.

## What is a Template and a Template Registry?

A template defines how an MDIO dataset is structured: names of dimensions and coordinates, the default variable name, chunking hints, and attributes to be stored. Since many seismic datasets share common structures (e.g., 3D post-stack, 2D post-stack, pre-stack CDP/shot, etc.), MDIO ships with a pre-populated template registry and APIs to fetch or register templates.

Fetching a template from it returns a copied instance you can freely customize without affecting others.

In [ ]:
from mdio.builder.template_registry import get_template
from mdio.builder.template_registry import get_template_registry
from mdio.builder.template_registry import list_templates

registry = get_template_registry()
registry  # pretty HTML in notebooks

We can list all registered templates and get a list as well.

In [ ]:
list_templates()

## Defining a Minimal Custom Template

To define a custom template, subclass `AbstractDatasetTemplate` and set:

- `_name`: a public name for the template
- `_dim_names`: names for each axis of your data variable (the last axis is the trace/time or trace/depth axis)
- `_physical_coord_names` and `_logical_coord_names`: optional additional coordinate variables to store along the spatial grid
- `_load_dataset_attributes()`: optional attributes stored at the dataset level

You can also override chunking via the `full_chunk_shape` property after you know your dataset sizes.

Below we create a very unusual shot–like template with two spatial dims `track` and `sequence`, plus a `time` axis. We will store "some" X/Y coordinates as physical coordinates and store `gun`/`shot_line` as logical indices.

In [ ]:
from mdio.builder.templates.base import AbstractDatasetTemplate


class CustomFieldTemplate(AbstractDatasetTemplate):
    """A custom template that has unusual dimensions and coordinates."""

    def __init__(self, data_domain: str = "time") -> None:
        super().__init__(data_domain)
        # Dimension order matters; the last dimension is the trace time (or depth) axis
        self._dim_names = ("track", "sequence", self.trace_domain)
        # Additional coordinates: these are added on top of dimension coordinates
        self._physical_coord_names = ("some_x", "some_y")
        self._logical_coord_names = ("gun", "shot_line")
        self._var_chunk_shape = (4, 4, 64)

    @property
    def _name(self) -> str:  # public name for the registry
        return "CustomFieldTime"

    def _load_dataset_attributes(self) -> dict:
        return {
            "description": "Minimal 2D post-stack style template for demos",
            "domain": self.trace_domain,
        }


CustomFieldTemplate()

## Registering the Custom Template

The registry returns a deep copy of the template on every fetch. To make the template discoverable by name, register it first, then retrieve it with `get_template`.

In [ ]:
from mdio.builder.template_registry import register_template

register_template(CustomFieldTemplate())
print("Registered:", "CustomFieldTime" in list_templates())

custom_template = get_template("CustomFieldTime")
custom_template

You can also set units at any time. For this demo we’ll set milliseconds for the time axis. The spatial units will be inferred from the SEG-Y binary header during ingestion, but we can override later if necessary.

In [ ]:
from mdio.builder.schemas.v1.units import LengthUnitModel
from mdio.builder.schemas.v1.units import TimeUnitModel

custom_template.add_units(
    {
        "time": TimeUnitModel(time="ms"),
        "some_x": LengthUnitModel(length="ft"),
        "some_y": LengthUnitModel(length="ft"),
    }
)
custom_template

## Changing chunk size (chunks) on an existing template

Often you will want to tweak the chunking strategy for performance. You can do this in two ways:

- When defining a subclass, set a default in the constructor (e.g., `self._var_chunk_shape = (...)`).
- On an existing template instance, assign to the `full_chunk_shape` property once you know your final
  dataset sizes (the tuple length must match the number of data dimensions).

Below is a tiny demo showing how to modify the chunk shape on a fetched template. We first build the
template with known sizes to satisfy validation, then update `full_chunk_shape`.

```{note}
In the SEG-Y to MDIO conversion workflow, MDIO infers the final grid shape from the SEG-Y headers. It’s
common to set or adjust `full_chunk_shape` right before calling `segy_to_mdio`, using the same sizes
you expect for the final array.
```

In [ ]:
_ = custom_template.build_dataset(name="demo-only", sizes=(4, 4, 128))
# pick smaller chunks than the full array for better parallelism and IO
custom_template.full_chunk_shape = (2, 2, 32)

custom_template

## Recap: Key APIs Used

- Template registry helpers: `get_template_registry`, `list_templates`, `register_template`, `get_template`
- Base template to subclass: `AbstractDatasetTemplate`

With these pieces, you can standardize how your seismic data is represented in MDIO and keep ingestion code concise and repeatable.
